In [62]:
import string
import re

In [61]:
name = 'PTV_5000'
# list_names = ['CTV_6000','CTV_7000','GTV','CTV_5600','PTV_7000-05','PTV!_6000-05',
#               'PTV_All','PTV','CTV','PTV_ALL','PTV70+PTV60','PTV60','PTV56','CTV60','CTV70','PTV!_Eval_6000','PTV2n',
#               'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb']
list_names = ['CT 60','GTV','CTV_5600','PTV_7000-05','PTV_All','PTV70+PTV60','PTV60','PTV!_Eval_6000','PTV2n',
              'CTVn_6000','CTVn','opt_CTV56_crop03','PTV^Physician1','GTV_2000^ICG','PTV1','GTVp1','GTVp2','PTVn','PTVsb']

# note, can easily check spaces and legnth

# to do: add exmples from the TG 263 paper

In [73]:
# note, this does not work for rule 10 of the guidelines 
# TO DO: split by ^ immediately (after length and space check) and disregard everything afterwards
# thought process for suffixes: go through and remove correct parts of name in the correct ordre. 
#                               if anything is leftover without a ^, then it is wrong

# TO DO: 
# E-PTV_Ev05_xxxx	E-PTV_Ev05_xxxx
# E-PTV_xxxx	E-PTV_xxxx

def check_target_compliance(target_name):
    
    #first check if there are spaces, then it's automatically not tg 263 compliant
    if ' ' in target_name:
        print(" spaces ")
        return False
    target_prefix = target_name.split("_")[0]
    target_suffix = target_name.replace(target_prefix,'')
    print("****")

    print("prefix", target_prefix)
#     print("suffix", target_suffix)

    is_correct = True
    reasoning = ''
  

    # CHECKING PREFIX
    list_allowed_prefixes = ['PTV!','GTV','CTV','ITV','IGTV','ICTV','PTV'] #rule 1
    list_allowed_classifiers = ['n', 'p', 'sb', 'par', 'v', 'vas',''] # rule 2, including none
    #note: ordering of the above mattes, PTV! sb before PTV, and '' should be last -- this is for when removing it from prefix
    
    if not target_prefix.startswith(tuple(list_allowed_prefixes)): # rule 1
        print("fails rule 1")
        return False
    
    for p in list_allowed_prefixes: 
        if target_prefix.startswith(p):
            target_prefix = target_prefix.replace(p,'')
            break
    print("After Rule 1:",target_prefix)

    
    
    
    if target_prefix != '':
        if target_prefix[0] == "^": # rule 9
            print("its true, all after ^")
            return True
        prefix_no_digits = target_prefix.rstrip(string.digits)
        print("prefixno digits:",prefix_no_digits)
        if not prefix_no_digits in list_allowed_classifiers: #rule 2
            print("fails rule 2")
        
        for c in list_allowed_classifiers:
            if target_prefix.startswith(c):
                target_prefix = target_prefix.replace(c,'')
                break
        print("After rule 2:",target_prefix)
        
        if target_prefix != '' and (len(target_prefix) > 1 or not target_prefix.isdigit()):
            # rule #3 --> could be tricky, saying len sb max 1 becuase i doubt the bunbers would go up to 10
            #but... there could be a random number there that isn't meant to represnet spatially distinct targets.
            print("fails rule 3")
            
    # TO DO -- check if ends with -xx for prefix eg CTVp2-05
    
    # CHECKING SUFFIX
    
    # TO DO: SKIPPING RULES 4 AND 5 FOR NOW
    # for rule 5, check thorugh TG again, however will need to do starts with i believe to accoutn for extra dose etc put at end
    # then need to remove it and analyse rest of structure as before
    # rule #6
    
    if target_suffix != '':
        relative_dose_suffixes = ["_High", "_Mid","_Low"]
        print("suffix", target_suffix)

        numerical_dose = False
        if target_suffix.startswith(tuple(relative_dose_suffixes)):
            #ok it is relavice dose
            # TO DO: add 01 02 etc
            for r in relative_dose_suffixes:
                if target_suffix.startswith(c):
                    target_suffix = target_suffix.replace(c,'')
                    break

            placeholder = 1

        elif bool(re.match(r'^_\d{4}', target_suffix)): #cGy
            target_suffix = target_suffix[5:]
            placeholder = 2
            numerical_dose = True
    #     elif : TO DO : Gy


        print("After rule 6: ", target_suffix)
        if numerical_dose: #rule 7, this is optional, only if the # fx are indicated it should be with an x
            fraction_pattern = r'^x\d{1,2}'
            if bool(re.match(fraction_pattern, target_suffix)):
                fractions = True
                target_suffix = re.sub(fraction_pattern, '', target_suffix)

            print("After rule 7: ", target_suffix)

        #rule 8
        if bool(re.match(r'^-\d{2}', target_suffix)):
            target_suffix = target_suffix[3:]

            print("after rule 8:", target_suffix)

        # rule 9
        if target_suffix == '' or target_suffix[0] == "^":
            print("ALL GOODDDDDD")
        else:
            print("FAILED~!")

            

In [72]:
for name in list_names:
    check_target_compliance(name)

 spaces 
****
prefix GTV
After Rule 1: 
****
prefix CTV
After Rule 1: 
suffix _5600
After rule 6:  
After rule 7:  
ALL GOODDDDDD
****
prefix PTV
After Rule 1: 
suffix _7000-05
After rule 6:  -05
After rule 7:  -05
after rule 8: 
ALL GOODDDDDD
****
prefix PTV
After Rule 1: 
suffix _All
After rule 6:  _All
FAILED~!
****
prefix PTV70+PTV60
After Rule 1: 70+60
prefixno digits: 70+
fails rule 2
After rule 2: 70+60
fails rule 3
****
prefix PTV60
After Rule 1: 60
prefixno digits: 
After rule 2: 60
fails rule 3
****
prefix PTV!
After Rule 1: 
suffix _Eval_6000
After rule 6:  _Eval_6000
FAILED~!
****
prefix PTV2n
After Rule 1: 2n
prefixno digits: 2n
fails rule 2
After rule 2: 2n
fails rule 3
****
prefix CTVn
After Rule 1: n
prefixno digits: n
After rule 2: 
suffix _6000
After rule 6:  
After rule 7:  
ALL GOODDDDDD
****
prefix CTVn
After Rule 1: n
prefixno digits: n
After rule 2: 
****
prefix opt
fails rule 1
****
prefix PTV^Physician1
After Rule 1: ^Physician1
its true, all after ^
****
prefi

In [4]:
name = "PTV"
name.split("_")

['PTV']

In [12]:
prefix = name.split("_")[0]

In [13]:
suffix = name.replace(prefix,'')

In [15]:
print("prefix",prefix)
print("suffix",suffix)

prefix PTV
suffix _5000


In [67]:
target_suffix = "x20x50"

fraction_pattern = r'^x\d{1,2}'
if bool(re.match(fraction_pattern, target_suffix)):
    fractions = True
    target_suffix = re.sub(fraction_pattern, '', target_suffix)


In [68]:
target_suffix

'x50'